In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from openai import OpenAI
client = OpenAI()

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_RvcAT9QAWZ3ji1hwA4Cc1JBW', created_at=1717019422, description=None, instructions='Sie sind ein experte in Sachen Normen und Standards im Bauwesen. Beantworten Sie mir die folgende Frage und sie MÜSSEN die Quelle der Informationen, also Kapitel und Abschnitt, ohne Ausnahmen angeben.\nDie Fragen beziehen sich auf die gegebenen Normen DIN EN 1991-1-3:2010-12 oder DIN EN 1991-1-3/NA:2019-04.\nBeantworten Sie die Fragen ausschließlich anhand der abgerufenen Informationen.', metadata={}, model='gpt-4o-mini', name='Norm Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=15, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format=ResponseFormatText(type='text'), temperature=0.01, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_g7Vc2wBDNvOV3EvpkSTWl2vb'])), top_p=1.0), Assistant(id='asst_mjcjLkctL1P

In [4]:
assistant = client.beta.assistants.retrieve(
    assistant_id='asst_RvcAT9QAWZ3ji1hwA4Cc1JBW'
)

In [10]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Wie bestimme ich die Schneelast auf einem Fachdach? Das Gebäude soll in München stehen und ist ungefähr 100m über NN."
    }
  ]
)

In [11]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(
    thread_id=thread.id, 
    run_id=run.id,
    ))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

Um die Schneelast auf einem Fachdach in München, das sich auf etwa 100 m über NN befindet, zu bestimmen, sind folgende Schritte und Berechnungen gemäß der Norm DIN EN 1991-1-3:2010-12 erforderlich:

1. **Ermittlung der charakteristischen Schneelast auf dem Boden (sk)**: Diese wird in der Regel aus regionalen Schneelastkarten entnommen. Für München liegt der charakteristische Wert typischerweise bei etwa 1,0 bis 2,0 kN/m², abhängig von den spezifischen lokalen Bedingungen.

2. **Berechnung der Schneelast auf dem Dach (s)**: Die Schneelast auf dem Dach wird mit der Formel berechnet:
   \[
   s = \mu_i \cdot C_e \cdot C_t \cdot s_k
   \]
   Hierbei sind:
   - \( \mu_i \) der Formbeiwert für Schneelasten,
   - \( C_e \) der Umgebungskoeffizient (in der Regel 1,0 für übliche Bedingungen),
   - \( C_t \) der Temperaturkoeffizient (in der Regel 1,0, es sei denn, es gibt besondere Umstände, die eine Anpassung erfordern),
   - \( s_k \) der charakteristische Wert der Schneelast auf dem Boden[0]

In [7]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id,
    include=["step_details.tool_calls[*].file_search.results[*].content"]
)

print(run_steps)

SyncCursorPage[RunStep](data=[RunStep(id='step_JPhxPYSsEAjya0Q6p4UcRlg7', assistant_id='asst_RvcAT9QAWZ3ji1hwA4Cc1JBW', cancelled_at=None, completed_at=1726928404, created_at=1726928403, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_KIDjlcFSztwIWHW6i9MU9aC9', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_Ao6XzuI8zH0VsXPdlyKy1bQ9'), type='message_creation'), thread_id='thread_QAw4Znrm72NYBLxJciw5A676', type='message_creation', usage=Usage(completion_tokens=95, prompt_tokens=11230, total_tokens=11325), expires_at=None), RunStep(id='step_4pg3USMiXsb13RCeRsiYHLuj', assistant_id='asst_RvcAT9QAWZ3ji1hwA4Cc1JBW', cancelled_at=None, completed_at=1726928403, created_at=1726928401, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_KIDjlcFSztwIWHW6i9MU9aC9', status='completed', step_details=ToolCallsStepDetails(tool_calls=[Fil

In [23]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id='thread_PleARa2XYiyDwR0pJ2gx90s0',
    run_id='run_gU3o9enyUnnWXxFQFZlAoLWk',
    include=["step_details.tool_calls[*].file_search.results[*].content"]
)

print(run_steps)

SyncCursorPage[RunStep](data=[RunStep(id='step_6yYX0KsyuFxRwuoEUrYMP4cg', assistant_id='asst_RvcAT9QAWZ3ji1hwA4Cc1JBW', cancelled_at=None, completed_at=1726769137, created_at=1726769126, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_gU3o9enyUnnWXxFQFZlAoLWk', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_OehtKDAgQgEJf83lVNtS5UnS'), type='message_creation'), thread_id='thread_PleARa2XYiyDwR0pJ2gx90s0', type='message_creation', usage=Usage(completion_tokens=495, prompt_tokens=14457, total_tokens=14952), expires_at=None), RunStep(id='step_CCEWhnPXKyUs6YSbGJzNoZYb', assistant_id='asst_RvcAT9QAWZ3ji1hwA4Cc1JBW', cancelled_at=None, completed_at=1726769126, created_at=1726769124, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_gU3o9enyUnnWXxFQFZlAoLWk', status='completed', step_details=ToolCallsStepDetails(tool_calls=[Fi

In [39]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id='thread_RhKDFKOkJsK0iotsp0N2O7T2',
    run_id='run_ypX8pAYy8ibdpwdO4eSnhABP',
    #include=["step_details.tool_calls[*].file_search.results[*].content"]
)

print(run_steps)

SyncCursorPage[RunStep](data=[RunStep(id='step_UFsaaPkuqUt1xPO7qVrmtrjp', assistant_id='asst_7VQGXcbxkAgXMYNLns3e5tTU', cancelled_at=None, completed_at=1726936731, created_at=1726936731, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_ypX8pAYy8ibdpwdO4eSnhABP', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_nwbbHyNhpYx5ledWFiKsOHgq'), type='message_creation'), thread_id='thread_RhKDFKOkJsK0iotsp0N2O7T2', type='message_creation', usage=Usage(completion_tokens=10, prompt_tokens=256, total_tokens=266), expires_at=None), RunStep(id='step_KSCPi88M7rcOZCMMNdG1lzGX', assistant_id='asst_7VQGXcbxkAgXMYNLns3e5tTU', cancelled_at=None, completed_at=1726936731, created_at=1726936729, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_ypX8pAYy8ibdpwdO4eSnhABP', status='completed', step_details=ToolCallsStepDetails(tool_calls=[CodeInt

In [22]:
print(run_steps.data)

[RunStep(id='step_JpRZVL9NZw8WNOBq7t38e3RH', assistant_id='asst_RvcAT9QAWZ3ji1hwA4Cc1JBW', cancelled_at=None, completed_at=1726757724, created_at=1726757711, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_FpQGOxilpJPn1I0Cmm2HsodL', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_vqxPWSBFWpodAsB86TeOBkmp'), type='message_creation'), thread_id='thread_ca0ftx64kkZ3VWFT11xu5Ys0', type='message_creation', usage=Usage(completion_tokens=506, prompt_tokens=14457, total_tokens=14963), expires_at=None), RunStep(id='step_yVCgwVEwsbXff2DInCJu1AZK', assistant_id='asst_RvcAT9QAWZ3ji1hwA4Cc1JBW', cancelled_at=None, completed_at=1726757711, created_at=1726757709, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_FpQGOxilpJPn1I0Cmm2HsodL', status='completed', step_details=ToolCallsStepDetails(tool_calls=[FileSearchToolCall(id='call_GmP

In [17]:
run_step = client.beta.threads.runs.steps.retrieve(
    thread_id=thread.id,
    run_id=run.id,
    step_id="step_JpRZVL9NZw8WNOBq7t38e3RH",
    include=["step_details.tool_calls[*].file_search.results[*].content"]
)

print(run_step)

RunStep(id='step_JpRZVL9NZw8WNOBq7t38e3RH', assistant_id='asst_RvcAT9QAWZ3ji1hwA4Cc1JBW', cancelled_at=None, completed_at=1726757724, created_at=1726757711, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_FpQGOxilpJPn1I0Cmm2HsodL', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_vqxPWSBFWpodAsB86TeOBkmp'), type='message_creation'), thread_id='thread_ca0ftx64kkZ3VWFT11xu5Ys0', type='message_creation', usage=Usage(completion_tokens=506, prompt_tokens=14457, total_tokens=14963), expires_at=None)


In [13]:
from wolframalpha import Client
import os
wolfram_alpha_appid = os.environ["WOLFRAM_ALPHA_APPID"]
wa = Client(wolfram_alpha_appid)

In [34]:
import nest_asyncio
nest_asyncio.apply()

problem_plain_text = "s_k = 1.25 * [0.19 + 0.19 * ((100 + 140) / 760)^2]"
res = wa.query(problem_plain_text)
res


{'@success': True,
 '@error': 'false',
 '@xml:space': 'preserve',
 '@numpods': '7',
 '@datatypes': '',
 '@timedout': '',
 '@timedoutpods': '',
 '@timing': '0.892',
 '@parsetiming': '0.46900000000000003',
 '@parsetimedout': 'false',
 '@recalculate': '',
 '@id': 'MSP7246125eh61b99gge4e600005ig617cahei2530e',
 '@host': 'https://www6b3.wolframalpha.com',
 '@server': '4',
 '@related': 'https://www6b3.wolframalpha.com/api/v1/relatedQueries.jsp?id=MSPa7247125eh61b99gge4e600005ci0i7f4hhcghbbf6430556393126808734',
 '@version': '2.6',
 '@inputstring': 's = 1.25 * [0.19 + 0.19 * ((100 + 140) / 760)^2]',
 'pod': [{'@title': 'Input',
   '@scanner': 'Identity',
   '@id': 'Input',
   '@position': 100.0,
   '@error': 'false',
   '@numsubpods': 1,
   'subpod': {'@title': '',
    'img': {'@src': 'https://www6b3.wolframalpha.com/Calculate/MSP/MSP7248125eh61b99gge4e6000034h9d488925eceae?MSPStoreType=image/gif&s=4',
     '@alt': 's = 1.25 (0.19 + 0.19 ((100 + 140)/760)^2)',
     '@title': 's = 1.25 (0.19 +

In [37]:
res['pod']

[{'@title': 'Input',
  '@scanner': 'Identity',
  '@id': 'Input',
  '@position': 100.0,
  '@error': 'false',
  '@numsubpods': 1,
  'subpod': {'@title': '',
   'img': {'@src': 'https://www6b3.wolframalpha.com/Calculate/MSP/MSP7248125eh61b99gge4e6000034h9d488925eceae?MSPStoreType=image/gif&s=4',
    '@alt': 's = 1.25 (0.19 + 0.19 ((100 + 140)/760)^2)',
    '@title': 's = 1.25 (0.19 + 0.19 ((100 + 140)/760)^2)',
    '@width': 233,
    '@height': 38,
    '@type': 'Default',
    '@themes': '1,2,3,4,5,6,7,8,9,10,11,12',
    '@colorinvertable': 'true',
    '@contenttype': 'image/gif'},
   'plaintext': 's = 1.25 (0.19 + 0.19 ((100 + 140)/760)^2)'},
  'expressiontypes': {'@count': '1', 'expressiontype': {'@name': 'Default'}}},
 {'@title': 'Results',
  '@scanner': 'Identity',
  '@id': 'Result',
  '@position': 200.0,
  '@error': 'false',
  '@numsubpods': 2,
  '@primary': True,
  'subpod': [{'@title': '',
    '@primary': True,
    'img': {'@src': 'https://www6b3.wolframalpha.com/Calculate/MSP/MSP72